In [4]:
# %cd ..
%cd /home/janneke/Documents/Master/Machine_Learning_in_Practice/HMS/MLiP_group_10_task1_HMS/

/home/janneke/Documents/Master/Machine_Learning_in_Practice/HMS/MLiP_group_10_task1_HMS


In [5]:
import pandas as pd
import numpy as np
from logging import getLogger, basicConfig, INFO
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

from datasets.raw_data_loader import CustomRawDataset
from generics.configs import DataConfig
from utils.evaluation_utis import score_kl_divergence

In [6]:
basicConfig(level=INFO)
logger = getLogger('main')
config = DataConfig()

In [7]:
dataset = CustomRawDataset(config, subset_sample_count=100, mode="train", cache=True)
dataset.print_summary()

2024-02-12 12:10:30,362 - utils.general_utils - INFO - Processing and caching new dataset
INFO:utils.general_utils:Processing and caching new dataset
2024-02-12 12:10:30,504 - utils.general_utils - INFO - train DataFrame shape: (100, 15)
INFO:utils.general_utils:train DataFrame shape: (100, 15)
2024-02-12 12:10:30,505 - utils.general_utils - INFO - Labels: ['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']
INFO:utils.general_utils:Labels: ['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']
100%|██████████| 17300/17300 [00:02<00:00, 6272.43it/s]
100it [00:02, 36.23it/s]
2024-02-12 12:10:33,348 - utils.general_utils - INFO - Dataset cached at ./data/cache/CustomRawDataset_100_train.npz
INFO:utils.general_utils:Dataset cached at ./data/cache/CustomRawDataset_100_train.npz


Dataset Summary:
Mode: train
Total Samples: 100
Unique Patients: 100
Unique EEGs: 100
Unique Spectrograms: 100
Label Distribution:
seizure_vote    145
lpd_vote         60
gpd_vote         46
lrda_vote        50
grda_vote        57
other_vote      258
dtype: int64

Vote Statistics:
        seizure_vote  lpd_vote  gpd_vote  lrda_vote  grda_vote  other_vote
mean        1.450000  0.600000  0.460000   0.500000   0.570000    2.580000
median      0.000000  0.000000  0.000000   0.000000   0.000000    0.000000
var         2.775253  4.525253  4.149899   1.141414   1.378889   25.114747
Probabilities Loaded: 100
Features Loaded: 100

Configuration Summary:
+---------------------+--------------------------------------------------------------------------------------------------------------------------------------+
| Configuration       | Value                                                                                                                                |
+---------------------+------

In [8]:
x_train, x_test, y_train, y_test = train_test_split(dataset.features_per_sample, dataset.lbl_probabilities, test_size=0.33, random_state=42)
y0, y1, y2, y3, y4, y5 = zip(*y_train)

Let multiple separate models regress on one label each

In [9]:
y_pred = np.zeros(y_test.shape)

for i, lbl_group in enumerate([y0, y1, y2, y3, y4, y5]):
	y_train_group = lbl_group
	model = LinearRegression()
	model.fit(x_train, y_train_group)
	y_pred_group = model.predict(x_test)
	y_pred[:,i] = y_pred_group

y_pred[y_pred < 0] = 0
y_pred_probabilities = y_pred / np.sum(y_pred, axis=1)[:,None]

In [11]:
submission = pd.DataFrame(y_pred_probabilities, columns=dataset.label_cols)
solution = pd.DataFrame(y_test, columns=dataset.label_cols)

# Need to be added for score calculation, but are removed at the start of the function
submission.insert(0, "id", range(len(submission)))
solution.insert(0, "id", range(len(solution)))

score_kl_divergence(solution=solution, submission=submission, row_id_column_name="id")

15.650261556623274